In [2]:
import pandas as pd
import os
import datetime
import regex as re

In [3]:
pwd()

'/Users/julia/Desktop/Thesis/Code Repository'

In [5]:
#del df

In [6]:
df = pd.read_csv('mapped.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2,3,5,9,11,13,14,15,16,35,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df['Project Number'] = df['Project Number'].astype('str') 
df['Fiscal Period Code'] = pd.to_datetime(df['Fiscal Period Code'], format='%Y%m%') 
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [8]:
len(df)

2179046

### The problem is the naming of the projects is inconsistent, but luckily the salescloudclientno were kept

In [6]:
#ADVCLIENT20354 =df[df['SalesCloudClientNo'] == 10238248.0]
#ADVCLIENT20354['Project Name'].unique()

In [ ]:
#df1['Project Name']= df1['Project Name'].str.replace(r'[0-9]+','')
#df1['Project Name']= df1['Project Name'].str.replace(r'-','')
#df1['Project Name']= df1['Project Name'].str.replace(r',','')

In [9]:
def repl(x):
    x = re.sub(r"[^\w\s]", "", str(x))      #Removes all punctuations
    x = re.sub(r"\d+", "", str(x))       #Replaces digits
    x = re.sub(r"\s+", " ", str(x))          #Removes double spaces with single space
    #x = x.strip().lower()                    #Trims the string
    return x

In [10]:
df['Cleaned Name'] = df['Project Name'].apply(repl)

### Now create dictionary from 2019 Project numbers and map back, compare length

In [11]:
len(df)

2179046

In [ ]:
#ADVCLIENT20354 =df[df['SalesCloudClientNo'] == 10238248.0]
#ADVCLIENT20354['Cleaned Name'].unique()

In [ ]:
#ADVCLIENT20354 =df[df['SalesCloudClientNo'] == 10239491.0]
#ADVCLIENT20354['Project Number'].unique()

In [7]:
df_1618=df[df['Fiscal Period Code'] < '2019-01-01']

In [8]:
df_1921=df[df['Fiscal Period Code'] > '2018-01-12'] #it actually doesn't matter wether we split or not, becuase project numbers were not recorded before 2019

In [59]:
len(df_1618), len(df_1921)

(1306653, 1295819)

### store the mapping in a new column and see whether there are differences later

In [12]:
# obtain unique project numbers with their corresponding names in a df, and save this into a dictionary
unique = (df.drop_duplicates(subset='Project Number', keep= 'first').reset_index(drop=True))

In [13]:
len(unique), len(df['Project Number'].unique())

(81383, 81383)

In [108]:
len(df_1921['Project Name'].unique()), len(df_1921['Cleaned Name'].unique())

(78938, 67322)

### We need to get a frame that contains 2019 -2021 where one number = one name

In [14]:
#df_nameswithmorethanonenumber=pd.DataFrame({'Project Name':df_nameswithmorethanonenumber.index, 'count':df_nameswithmorethanonenumber.values})
#unique came from only 2019+ data
df_combo=unique.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project Name').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
df_nameswithmorethanonenumber=dfdf_combo[dfdf_combo['count'] != 1]
df_nameswithmorethanonenumber.reset_index(inplace=True)
df_nameswithmorethanonenumber.sort_values('count', 
                      inplace=True, 
                      ascending=False)

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [15]:
df_nameswithmorethanonenumber

,Project Name,count
386,ACCRUED FEES,5093
1240,COST TRANSFER,5084
1737,DORMANT PROJECT EXP,4932
4879,REVENUE GROUP PROJECT,214
0,#XL_EVAL_ERROR#,56
...,...,...
2033,Elmhurst ROW,2
2032,Elgin MGP 2018,2
2031,Elgin MGP,2
2030,Elgin HQ Disposal,2


####
The problem is that there are several names which have more than one project number - This is because either there is a notation error, or they actualy started several projects while the system was transitioned. We have to exclude those for the mapping phase, but they can probably still be used in the overall analysis as they have unique project numbers
For now I remove the dots, and then after that map nan columns to project numbers based on project name.


In [17]:
df_toomanyprojects=df_nameswithmorethanonenumber[df_nameswithmorethanonenumber['count'] > 8]
len(df_toomanyprojects) 
df_toomanyprojects['Project Name']

386                ACCRUED FEES
1240              COST TRANSFER
1737        DORMANT PROJECT EXP
4879      REVENUE GROUP PROJECT
0               #XL_EVAL_ERROR#
4878              REVENUE GROUP
2445           GE Corp Env Prog
5740             Vandenberg AFB
1738    DORMANT PROJECT EXPENSE
4968      Revenue Group Project
Name: Project Name, dtype: object

In [18]:
df_removepunct = dfdf_combo[(dfdf_combo['count'] < 9)] 
df_removepunct.reset_index(inplace=True)
df_removepunct.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_removepunct

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Project Name,count
35598,Itasca TSS101,8
47245,Phase I Environmental Site Ass,8
49434,SALES DISCOUNTS,8
19499,Cintas Corporation,8
14039,CBRE-BOA-NH2 147-PLAISTOW NH-D,8
...,...,...
20562,ComEd WO#15392278 5611 N Kimball Soil Sample a...,1
20563,ComEd WO#15453125 Hamilton Morse Soil Sample a...,1
20564,ComEd WO#15467397 W Glenlake a,1
20565,ComEd WO#15513724 Stony Island,1


In [28]:
#remove these names and create a new df1 or should I only be using df_1921 here? Here I just ignore all the values that have more than 7 different ids per name
df_mergebacklater = df[df['Project Name'].isin(df_toomanyprojects['Project Name'])]

In [29]:
df1 = df[~df['Project Name'].isin(df_toomanyprojects['Project Name'])]

In [26]:
#df1 = df_mergebacklater[df_mergebacklater['Project Name'].isin(df_removepunct['Project Name'])] #so df1 was just created to have the project numbers without punctuation

In [32]:
len(df1) 

1621252

In [31]:
len(df_mergebacklater)

557794

In [33]:
df1['Project Number']= df1['Project Number'].str.replace(r'.','') #df["new_column"] = df['review'].str.replace('[^\w\s]','')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df1['Project Number'].nunique()

60634

In [35]:
df1.head()

,Fiscal Period Code,Project Number,Project Id,Project Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_current,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,Cleaned Name
0,1970-01-01 00:00:00.000201601,nan,ADVCLIENT5042,UNIVAR - PROJECT MONARCH,NaN,NaN,Open/Active,10239939.0,NaN,Time and Material,...,-23.04,-18.82,-23.04,3.28,2.93,3.28,1432.73,158.47,I,UNIVAR PROJECT MONARCH
1,1970-01-01 00:00:00.000201601,nan,MPICLIENT000410000080310,MP-77B Pine Hill PS - Gen Srvcs,NaN,NaN,Open/Active,10238459.0,NaN,Lump Sum,...,3.19,-49.78,3.19,3.03,0.00,3.08,0.00,172.62,G,MPB Pine Hill PS Gen Srvcs
2,1970-01-01 00:00:00.000201601,nan,mdKCAUGHHORN220630032443,YJWWTP Post-Sandy Rehab- PhIII,NaN,NaN,Open/Active,10238060.0,NaN,Cost plus Fixed Fee,...,1.75,-2243.68,1.75,2.99,2.99,2.99,997.60,156.56,I,YJWWTP PostSandy Rehab PhIII
3,1970-01-01 00:00:00.000201601,nan,ADVCLIENT22208,CIP PM Reporting Dashboard,NaN,NaN,Open/Active,10238090.0,NaN,Time and Material,...,13.19,-1764.53,13.19,3.92,0.00,3.22,0.00,183.72,I,CIP PM Reporting Dashboard
4,1970-01-01 00:00:00.000201601,nan,ADVCLIENT20354,District Wide Solids Master Plan,NaN,NaN,Open/Active,10238114.0,NaN,Lump Sum,...,-79.21,-86.61,-79.21,2.78,2.80,2.78,19404.79,161.48,I,District Wide Solids Master Plan


In [135]:
df1[df1['Project Name']== 'SCA_96368_La Canada']['Project Number'].unique()

array(['nan', 'ASRTM0KB6368', 'GWR000KB6368'], dtype=object)

In [36]:
df_combo=df1.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project Name').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
#df_nameswithmorethanonenumber=dfdf_combo[dfdf_combo['count'] != 1]
dfdf_combo.reset_index(inplace=True)
dfdf_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)

dfdf_combo

,Project Name,count
32337,Cintas Corporation,9
87075,SALES DISCOUNTS,9
57938,INTEROFFICE LABOR TRANSFER,9
57937,INTEROFFICE LABOR COST TRANSFER,8
35942,Curtis Specialty Papers Site,8
...,...,...
45408,FMC MainSt_GloucesterVA_Survey,1
10802,ARC01093.1000 ERA Support,1
10803,ARC01093.1000 GE ERA Support,1
45405,FMC MacClennyFL_REM&Restoratio,1


In [40]:
len(df_notmap2)

9739

In [37]:
#Okey so those that have even more than two numbers now, these seem to be impossible to map. Those with 2 have nan values left, so I replace those
df_notmapable = dfdf_combo[dfdf_combo['count'] > 3]
df_notmap2 = df[df['Project Name'].isin(df_notmapable['Project Name'])] 

 # those are unmapable as well, so we merge them together
df_cannotmap = pd.concat([df_mergebacklater, df_notmap2])
df_cannotmap['Project Number'].nunique() # 15444 cannot be mapped => should be removed from dataframe later

15905

In [38]:
len(df_cannotmap)

567533

In [139]:
df_cannotmap.to_csv('morethan3.csv') 

In [23]:
len(df_notmap) + len(df_notmap2), len(df_cannotmap)

(565707, 565707)

In [ ]:
df2 = df[~df['Project Name'].isin(df_cannotmap['Project Name'])] 

In [ ]:
df_combo=df1.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project Name').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
#df_nameswithmorethanonenumber=dfdf_combo[dfdf_combo['count'] != 1]
dfdf_combo.reset_index(inplace=True)
dfdf_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)

dfdf_combo

In [140]:
## Those that can still be saved
#df_3 = df_nameswithmorethanonenumber[df_nameswithmorethanonenumber['count'] == 3]
df_three=dfdf_combo[dfdf_combo['count'] < 4]
df_three.reset_index(inplace=True)
df_three.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_three

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,index,Project Name,count
0,18812,Calumet TSS150,3
444,45283,OU3 On Call Consulting Service,3
468,21507,Continental Park,3
469,43191,NJ-04339 (10077),3
470,43192,NJ-05179 (30469),3
...,...,...,...
31090,37,00148007.0000-Big Island 122.21r Reports,1
31091,59308,Y0335135.1828-NCA 76 351828 Year 3,1
31092,71,00228806.0000-Columbus 2017 - COS Studies,1
31093,70,00228805.0001 Col HCWP Hypochlorite Design,1


In [141]:
df1[df1['Project Name'] == 'interoffice transfer']['Project Number'].unique()

array(['nan', 'AZ000YYYAD10', 'CO000YYYAD10'], dtype=object)

### Now we create a df and a dict to map back one number on nans, but first we need to make sure that those we pick are really only nanas

In [142]:
df_double = df1[df1['Project Name'].isin(df_three['Project Name'])]
df_double2 = df_double[df_double['Project Number'].notna()]
df_double2 = df_double2[df_double2['Project Number'] != 'nan']

In [145]:
df_double2[df_double2['Project Name'] == 'interoffice transfer']['Project Number'].unique()

array(['AZ000YYYAD10', 'CO000YYYAD10'], dtype=object)

In [144]:
df_combo=df_double2.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project Name').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
df_nameswithmorethanonenumber=dfdf_combo[dfdf_combo['count'] != 1]
df_nameswithmorethanonenumber.reset_index(inplace=True)
df_nameswithmorethanonenumber.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_nameswithmorethanonenumber

#okey so there are still some more projects with the same name and two ids

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Project Name,count
303,FDOT - TWO7 - SR525 S Orange A,3
892,WSIP - T&M,3
231,Continued Support for PG Reuse Petition,3
13,2019 Environmental Compliance,3
452,JLL BOA TX2-837 Bedford TX IAQ,3
...,...,...
310,FMC Carteret Remed and Radio,2
311,FORA: TECHNICAL & SITE SERVICE,2
312,FORMER AGT SURFACE IMPOUNDMENT,2
313,FORMER ATKEMIX LOUISVILLE SITE,2


In [146]:
## create the the dataframe, that still has double projects
df_3or2 = df[df['Project Name'].isin(df_nameswithmorethanonenumber['Project Name'])]

In [147]:
df_3or2.to_csv('df_3or2.csv')

In [148]:
df_final = dfdf_combo[dfdf_combo["count"] == 1]
df_final.reset_index(inplace=True)
df_final.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_final

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Project Name,count
0,"""Annuitant"" On-Site Support -",1
38962,Lord Saegertown Phase 1 PIP Separation,1
38964,Lorentz Barrel and Drum Site,1
38965,LoriAl D140 2018 Monitoring,1
38966,Los Alamos Nat'l Lab - Amendme,1
...,...,...
19488,Clorox Chicago 2021,1
19489,Closed Landfill Corrective Mea,1
19490,Closed Project Expenses,1
19491,Closner Substation Septic Tank,1


In [149]:
#Now why is there a difference between df_final and df_double2 in length?
df_idk = df_double2[df_double2['Project Name'].isin(df_final['Project Name'])]
len(df_idk)

667872

In [150]:
df_idk.tail()

,Fiscal Period Code,Project Number,Project Id,Project Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
2179041,2021-03-01,30085651,3.00004e+14,FDOT-TWO 158 SR 152 (Baymeadows Rd) at Sanchez...,Plan Required,NaN,New,NaN,NaN,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FDOTTWO SR Baymeadows Rd at Sanchez Road
2179042,2021-03-01,30085652,3.00004e+14,UPEE2117-Yermo Fueling Survey,Plan Required,NaN,New,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,UPEEYermo Fueling Survey
2179043,2021-03-01,30085657,3.00004e+14,Cintas Oswego St. Facility,Plan Signed Off,NaN,New,NaN,30081650,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Cintas Oswego St Facility
2179044,2021-03-01,30085660,3.00004e+14,BKK Additional Phase 1 Design Services,Plan Required,NaN,New - Pending Approval,NaN,30078666,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BKK Additional Phase Design Services
2179045,2021-03-01,30085661,3.00004e+14,City of Baltimore 1267E TO 2 Water Design & En...,Plan Required,NaN,New,NaN,NaN,Cost plus Fixed Fee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,City of Baltimore E TO Water Design Engineerin...


In [151]:
df_idk['Project Name'].nunique()

58455

In [153]:
df_nomoredoubles = (df_idk.drop_duplicates(subset='Project Name', keep= 'first').reset_index(drop=True))
len(df_nomoredoubles)

58455

In [154]:
df_nomoredoubles = df_nomoredoubles[['Project Name', 'Project Number']].copy()
df_nomoredoubles.columns = ['Project_Name', 'Project_Number']
df_nomoredoubles

,Project_Name,Project_Number
0,Yakima Site Walk and Well Loca,WAYK20180001
1,SCA 76 351605 Year 3,Y03351351605
2,Multi 1.04.04,PSF000000012
3,Braidwood GPR,EG0137280000
4,"37-049 - 651 ALBRIGHT AVENUE, YORK, PA",B00858510087
...,...,...
58450,FDOT-TWO 158 SR 152 (Baymeadows Rd) at Sanchez...,30085651
58451,UPEE2117-Yermo Fueling Survey,30085652
58452,Cintas Oswego St. Facility,30085657
58453,BKK Additional Phase 1 Design Services,30085660


In [155]:
df1 = df1.rename(columns={'Project Name': 'Project_Name', 'Project Number':'Project_Number'})

In [156]:
#sub_df1 = df1[['Project Name', 'Project Number']].copy()
#sub_df1.columns = ['Project_Name', 'Project_Number']
#sub_df1

In [157]:
realData = pd.Series(data=df_nomoredoubles.Project_Number.values, index=df_nomoredoubles.Project_Name)
df1['Project_Number'] = df1.Project_Name.map(realData).fillna(df1['Project_Number'])

In [159]:
df1[df1['Project_Name'] == 'PPG Barberton Lime Lake 6 Pre-Design Inv']['Project_Number'].unique()

array(['OH0025020002'], dtype=object)

In [160]:
len(df1)

924174

In [161]:
df1

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
0,2016-01-01,nan,ADVCLIENT5042,UNIVAR - PROJECT MONARCH,NaN,NaN,Open/Active,10239939.0,NaN,Time and Material,...,-18.82,-23.04,3.28,2.93,3.28,1432.73,158.47,I,1306653,UNIVAR PROJECT MONARCH
2,2016-01-01,30001238,mdKCAUGHHORN220630032443,YJWWTP Post-Sandy Rehab- PhIII,NaN,NaN,Open/Active,10238060.0,NaN,Cost plus Fixed Fee,...,-2243.68,1.75,2.99,2.99,2.99,997.60,156.56,I,1306653,YJWWTP PostSandy Rehab PhIII
9,2016-01-01,30001347,MPICLIENT002660000080310,Abandoned Chemical Sales Fac.,NaN,NaN,Open/Active,10238193.0,NaN,D,...,-5079.36,0.47,2.94,2.97,2.95,771.86,156.56,I,1306653,Abandoned Chemical Sales Fac
10,2016-01-01,30001355,MPICLIENT002660000080310,NYSDEC Jimmy's RA,NaN,NaN,Open/Active,10238193.0,NaN,Cost plus Fixed Fee,...,187.72,-51.96,2.97,3.04,2.96,4225.29,158.47,I,1306653,NYSDEC Jimmys RA
11,2016-01-01,30001367,MPICLIENT002660000080310,Chemical Sales RD WA 34,NaN,NaN,Open/Active,10238193.0,NaN,D,...,140.12,0.00,3.01,3.29,3.01,930.61,156.56,I,1306653,Chemical Sales RD WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179041,2021-03-01,30085651,3.00004e+14,FDOT-TWO 158 SR 152 (Baymeadows Rd) at Sanchez...,Plan Required,NaN,New,NaN,NaN,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FDOTTWO SR Baymeadows Rd at Sanchez Road
2179042,2021-03-01,30085652,3.00004e+14,UPEE2117-Yermo Fueling Survey,Plan Required,NaN,New,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,UPEEYermo Fueling Survey
2179043,2021-03-01,30085657,3.00004e+14,Cintas Oswego St. Facility,Plan Signed Off,NaN,New,NaN,30081650,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Cintas Oswego St Facility
2179044,2021-03-01,30085660,3.00004e+14,BKK Additional Phase 1 Design Services,Plan Required,NaN,New - Pending Approval,NaN,30078666,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BKK Additional Phase Design Services


In [174]:
df_wait = df[df['Project Name'].isin(df_toomanyprojects['Project Name'])] 
len(df_wait)

557794

In [187]:
df_wait = df_wait.rename(columns={'Project Name': 'Project_Name', 'Project Number':'Project_Number'})

In [324]:
frames = [df1, df_wait]

result = pd.concat(frames)

In [189]:
len(result), len(df1) + len(df_wait)

(1481968, 1481968)

In [190]:
df1.to_csv('df1.csv') 

In [191]:
result

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
0,2016-01-01,nan,ADVCLIENT5042,UNIVAR - PROJECT MONARCH,NaN,NaN,Open/Active,10239939.0,NaN,Time and Material,...,-18.82,-23.04,3.28,2.93,3.28,1432.73,158.47,I,1306653,UNIVAR PROJECT MONARCH
2,2016-01-01,30001238,mdKCAUGHHORN220630032443,YJWWTP Post-Sandy Rehab- PhIII,NaN,NaN,Open/Active,10238060.0,NaN,Cost plus Fixed Fee,...,-2243.68,1.75,2.99,2.99,2.99,997.60,156.56,I,1306653,YJWWTP PostSandy Rehab PhIII
9,2016-01-01,30001347,MPICLIENT002660000080310,Abandoned Chemical Sales Fac.,NaN,NaN,Open/Active,10238193.0,NaN,D,...,-5079.36,0.47,2.94,2.97,2.95,771.86,156.56,I,1306653,Abandoned Chemical Sales Fac
10,2016-01-01,30001355,MPICLIENT002660000080310,NYSDEC Jimmy's RA,NaN,NaN,Open/Active,10238193.0,NaN,Cost plus Fixed Fee,...,187.72,-51.96,2.97,3.04,2.96,4225.29,158.47,I,1306653,NYSDEC Jimmys RA
11,2016-01-01,30001367,MPICLIENT002660000080310,Chemical Sales RD WA 34,NaN,NaN,Open/Active,10238193.0,NaN,D,...,140.12,0.00,3.01,3.29,3.01,930.61,156.56,I,1306653,Chemical Sales RD WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951434,2019-08-01,CXPP1829.FCI0,NaN,#XL_EVAL_ERROR#,NaN,NaN,Closed,10239458.0,NaN,Lump Sum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,XL_EVAL_ERROR
1951435,2019-09-01,CXPP1829.FCI0,NaN,#XL_EVAL_ERROR#,NaN,NaN,Closed,10239458.0,NaN,Lump Sum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,XL_EVAL_ERROR
1951436,2019-10-01,CXPP1829.FCI0,NaN,#XL_EVAL_ERROR#,NaN,NaN,Closed,10239458.0,NaN,Lump Sum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,XL_EVAL_ERROR
1951437,2019-11-01,CXPP1829.FCI0,NaN,#XL_EVAL_ERROR#,NaN,NaN,Closed,10239458.0,NaN,Lump Sum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,XL_EVAL_ERROR


### df_final does not conatin the project numbers with more than 8 mappings

In [ ]:
import numpy as np
#df1['Project Number']= df1['Project Number'].replace(r'^(nan)$', np.NaN, inplace=True) 
#df1['Project Number']= df1['Project Number'].str.replace(r'^(nan)$','NaN')
#leg3 = df1.combine(df_nomoredoubles,  overwrite=True)
#df1['Project Name']= df1['Project Name'].map(numbers_names_dict).fillna(df1['Project Name'])
#df1.update(df_idk)
#df1[df1['Project Name'] == 'PPG Barberton Lime Lake 6 Pre-Design Inv']['Project Number'].unique()
#leg3 = df1.combine(df_nomoredoubles,  overwrite=True)
#df1['Project Name']= df1['Project Name'].map(numbers_names_dict).fillna(df1['Project Name'])

In [84]:
#df_cannotmap.head()

In [85]:
#df_3or2.head()

### Next we subset from the overall dataframe the ones that have one number per project name, extract those, and then check which ones of those have cleaned names
we want to make sure that there is absolutely no chance we merge projects based on the fact that suddenly the cleaned name is the same. Hence, we only consider those projects that have a unique projet_name and project_number combination, but more than 1 project_number on Cleaned Names. However this can still happen if we have projects that are distinguished through numbers in their names

In [216]:
df_combo=result.groupby([ "Project_Name", "Project_Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project_Name').count()
dfdf_combo.pop('Project_Number')


#This should be the names with more than two numbers
dfdf_combo.reset_index(inplace=True)
dfdf_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)

dfdf_combo

,Project_Name,count
5922,ACCRUED FEES,5094
16437,COST TRANSFER,5085
23991,DORMANT PROJECT EXP,4933
48092,REVENUE GROUP PROJECT,215
3,#XL_EVAL_ERROR#,56
...,...,...
19944,"Columbus GACOL, GA 2020 SWPPP Q3",1
19945,"Columbus GACOL, GA SWPPP Q2",1
19946,Columbus OH 2020,1
19947,Columbus Orion PMIS Support Renewal 2,1


In [217]:
map_names = dfdf_combo[dfdf_combo['count'] == 1] 

In [218]:
df_mapnames = df1[df1['Project_Name'].isin(map_names['Project_Name'])] 
df_mapnames

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
0,2016-01-01,nan,ADVCLIENT5042,UNIVAR - PROJECT MONARCH,NaN,NaN,Open/Active,10239939.0,NaN,Time and Material,...,-18.82,-23.04,3.28,2.93,3.28,1432.73,158.47,I,1306653,UNIVAR PROJECT MONARCH
2,2016-01-01,30001238,mdKCAUGHHORN220630032443,YJWWTP Post-Sandy Rehab- PhIII,NaN,NaN,Open/Active,10238060.0,NaN,Cost plus Fixed Fee,...,-2243.68,1.75,2.99,2.99,2.99,997.60,156.56,I,1306653,YJWWTP PostSandy Rehab PhIII
9,2016-01-01,30001347,MPICLIENT002660000080310,Abandoned Chemical Sales Fac.,NaN,NaN,Open/Active,10238193.0,NaN,D,...,-5079.36,0.47,2.94,2.97,2.95,771.86,156.56,I,1306653,Abandoned Chemical Sales Fac
10,2016-01-01,30001355,MPICLIENT002660000080310,NYSDEC Jimmy's RA,NaN,NaN,Open/Active,10238193.0,NaN,Cost plus Fixed Fee,...,187.72,-51.96,2.97,3.04,2.96,4225.29,158.47,I,1306653,NYSDEC Jimmys RA
11,2016-01-01,30001367,MPICLIENT002660000080310,Chemical Sales RD WA 34,NaN,NaN,Open/Active,10238193.0,NaN,D,...,140.12,0.00,3.01,3.29,3.01,930.61,156.56,I,1306653,Chemical Sales RD WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179041,2021-03-01,30085651,3.00004e+14,FDOT-TWO 158 SR 152 (Baymeadows Rd) at Sanchez...,Plan Required,NaN,New,NaN,NaN,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FDOTTWO SR Baymeadows Rd at Sanchez Road
2179042,2021-03-01,30085652,3.00004e+14,UPEE2117-Yermo Fueling Survey,Plan Required,NaN,New,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,UPEEYermo Fueling Survey
2179043,2021-03-01,30085657,3.00004e+14,Cintas Oswego St. Facility,Plan Signed Off,NaN,New,NaN,30081650,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Cintas Oswego St Facility
2179044,2021-03-01,30085660,3.00004e+14,BKK Additional Phase 1 Design Services,Plan Required,NaN,New - Pending Approval,NaN,30078666,Lump Sum with IFRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BKK Additional Phase Design Services


#### then next 
we want to see whether we can map some projects together by merging stripping the names. For this step I also exlude the df_cannotmap and df_3or2, because did not have unique number name combinations before, so cant map that

In [219]:
#Cleaned names count, but first we need to map unique unclean names before we can map cleaned names
df_clean=df_mapnames.groupby([ "Cleaned Name", "Project_Number"] ).size().to_frame(name = 'count').reset_index()
df_clean=df_clean.groupby('Cleaned Name').count()
df_clean.pop('Project_Number')


#This should be the names with more than two numbers
df_clean.reset_index(inplace=True)
df_clean.sort_values('count', 
                      inplace=True, 
                      ascending=False)

In [ ]:
#df_clean[df_clean['count'] == 3].head(50)

In [251]:
x ='GP14JBMDOU06'
len(x)

12

In [269]:
df_mapnames[df_mapnames['Project_Number'] == 'ARC20985.4002']

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name


In [255]:
df_mapnames[df_mapnames['Cleaned Name'] == 'GPQCloseout']

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
1610996,2019-07-01,30012813,NaN,GP110003.Q003-Closeout,Plan Not Required,NaN,Closed-Open to Collect,10239055.0,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1610997,2019-08-01,30012813,NaN,GP110003.Q003-Closeout,Plan Not Required,NaN,Closed-Open to Collect,10239055.0,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1610998,2019-09-01,30012813,NaN,GP110003.Q003-Closeout,Plan Not Required,NaN,Closed-Open to Collect,10239055.0,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1610999,2019-10-01,30012813,NaN,GP110003.Q003-Closeout,Plan Not Required,NaN,Closed-Open to Collect,10239055.0,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1611000,2019-11-01,30012813,NaN,GP110003.Q003-Closeout,Plan Not Required,NaN,Closed-Open to Collect,10239055.0,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902951,2020-11-01,30010239,3.00001e+14,GP001087.Q001-Closeout,Plan Not Required,NaN,Closed,NaN,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1902952,2020-12-01,30010239,3.00001e+14,GP001087.Q001-Closeout,Plan Not Required,NaN,Closed,NaN,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1902953,2021-01-01,30010239,3.00001e+14,GP001087.Q001-Closeout,Plan Not Required,NaN,Closed,NaN,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout
1902954,2021-02-01,30010239,3.00001e+14,GP001087.Q001-Closeout,Plan Not Required,NaN,Closed,NaN,NaN,Revenue group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,GPQCloseout


### So we need to find a way to extract the legacy number from the name, then put it into another column, remove the dot, create a dict of new and legacy number
### and then map it back to together
- what are the signs that the numbers cloud match?
- Salescloudclientno (but that can be missing sometimes for the new projects, so not reliable)
- we can split be the timeframe, only projects at and after 2019.07 will have the legacy number in the name
- the new project number starts with 3 for sure
- the old legacy number has 13 random characters, with a . at 8th character and a - at 14th character 

In [278]:
df2019 = df_mapnames[df_mapnames['Fiscal Period Code'] >' 2019.06.01']
df2019 = (df2019.drop_duplicates(subset='Project_Name', keep= 'first').reset_index(drop=True))
len(df2019)

40281

In [279]:
df2019['Project_Name'] = df2019['Project_Name'].astype('str') 


In [280]:
df2019[['Legacy', 'Name_remainder']] = df2019['Project_Name'].str.split('-', 1, expand=True)
df2019['Legacylength'] = df2019['Legacy'].str.len()
df13 = df2019[df2019['Legacylength'] == 13]
df13['legacycleaned']= df13['Legacy'].str.replace(r'.','')
df13['Legacylengthcleaned'] = df13['legacycleaned'].str.len()
df12 = df13[df13['Legacylengthcleaned'] == 12]
df12

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,overhead_current,overhead_rate,revenue_type,count,Cleaned Name,Legacy,Name_remainder,Legacylength,legacycleaned,Legacylengthcleaned
9,2019-07-01,30011527,NaN,DEMCLOE0.3000-CVX_CE_WV_123000_RAINELLE,Plan Required,NaN,Open/Active,10238092.0,NaN,Time and Material,...,NaN,NaN,NaN,21,DEMCLOECVX_CE_WV__RAINELLE,DEMCLOE0.3000,CVX_CE_WV_123000_RAINELLE,13,DEMCLOE03000,12
10,2019-07-01,30011528,NaN,GP09BPNA.M046-MI-05614,Plan Not Required,NaN,Closed-Open to Collect,10238206.0,NaN,Revenue group,...,NaN,NaN,NaN,21,GPBPNAMMI,GP09BPNA.M046,MI-05614,13,GP09BPNAM046,12
11,2019-07-01,30011524,NaN,DEMCLOE0.2770-CVX_CE_WV_122770_MILTON,Plan Required,NaN,Open/Active,10238092.0,NaN,Time and Material,...,NaN,NaN,NaN,21,DEMCLOECVX_CE_WV__MILTON,DEMCLOE0.2770,CVX_CE_WV_122770_MILTON,13,DEMCLOE02770,12
12,2019-07-01,30011525,NaN,GP09BPNA.M045-MI-05631,Plan Not Required,NaN,Closed-Open to Collect,10238206.0,NaN,Revenue group,...,NaN,NaN,NaN,21,GPBPNAMMI,GP09BPNA.M045,MI-05631,13,GP09BPNAM045,12
13,2019-07-01,30011526,NaN,HT128931.0000-JLL BOA CA2-182 Morgan Hill CIS,Plan Required,NaN,Open/Active,10239479.0,NaN,Lump Sum with IFRS,...,NaN,NaN,NaN,21,HTJLL BOA CA Morgan Hill CIS,HT128931.0000,JLL BOA CA2-182 Morgan Hill CIS,13,HT1289310000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40000,2021-03-01,30084238,3.00004e+14,JK018001.0003-SJCNTY - Winton Utility Design,Plan Required,NaN,Open/Active,NaN,30084235,Time and Material with cap,...,NaN,NaN,NaN,1,JKSJCNTY Winton Utility Design,JK018001.0003,SJCNTY - Winton Utility Design,13,JK0180010003,12
40025,2021-03-01,30084355,3.00004e+14,03029036.0004-UB Implementation Ph 2 BA Support,Plan Required,NaN,Open/Active,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,1,UB Implementation Ph BA Support,03029036.0004,UB Implementation Ph 2 BA Support,13,030290360004,12
40088,2021-03-01,30084671,3.00004e+14,WF0590PE.0337-PER-5A-101- Emerald Dunes Golf C...,Plan Required,NaN,Open/Active,NaN,NaN,Time and Material,...,NaN,NaN,NaN,1,WFPEPERA Emerald Dunes Golf Course Hole Mod,WF0590PE.0337,PER-5A-101- Emerald Dunes Golf Course Hole 16 ...,13,WF0590PE0337,12
40140,2021-03-01,30084975,3.00004e+14,TL011667.0001-UPL/Flupyrimin TFD Study,Plan Required,NaN,Open/Active,NaN,30084946,Lump Sum with IFRS,...,NaN,NaN,NaN,1,TLUPLFlupyrimin TFD Study,TL011667.0001,UPL/Flupyrimin TFD Study,13,TL0116670001,12


In [282]:
## so there are still some projects which don't have a uniquely new project ID, we first need to map those to be 100% certain
df12['legacycleaned'].nunique()

14509

In [284]:
#Cleaned names count, but first we need to map unique unclean names before we can map cleaned names
df_clean=df12.groupby([ "legacycleaned", "Project_Number"] ).size().to_frame(name = 'count').reset_index()
df_clean=df_clean.groupby('legacycleaned').count()
df_clean.pop('Project_Number')


#This should be the names with more than two numbers
df_clean.reset_index(inplace=True)
df_clean.sort_values('count', 
                      inplace=True, 
                      ascending=False)
df_clean

,legacycleaned,count
13935,UPEE20060005,2
12130,NYNG2019TS22,2
13053,SM0103090014,2
5198,CA0007040012,2
6401,DEMARNE04631,2
...,...,...
4841,BB0200530002,1
4842,BB0200690001,1
4843,BB0201180000,1
4844,BB0201452017,1


In [290]:
Legacynumber_double = df_clean[df_clean['count'] == 2]
Legacynumber_double['legacycleaned'].nunique()
Legacynumber_double.to_csv('Legacynumber_double.csv')

In [291]:
unique_legacy =df_clean[df_clean['count'] == 1]
df12_uniquelegacy = df12[df12['legacycleaned'].isin(unique_legacy['legacycleaned'])]

In [292]:
len(df12), len(df12_uniquelegacy)

(14522, 14496)

In [296]:
legacydict = dict(zip(df12_uniquelegacy['legacycleaned'], df12_uniquelegacy['Project_Number']))

In [ ]:
#legacydict #DEMCLOE03000': '30011527',

In [313]:
result[result['Project_Number'] == '30018361']

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
16955,2016-01-01,30018361,ADVCLIENT2462,TOTAL/RETIA - Pester Burn Pond,NaN,NaN,Open/Active,10239122.0,NaN,Time and Material,...,0.00,0.00,2.44,2.55,2.44,837.79,151.95,I,1306653,TOTALRETIA Pester Burn Pond
50701,2016-02-01,30018361,ADVCLIENT2462,TOTAL/RETIA - Pester Burn Pond,NaN,NaN,Open/Active,10239122.0,NaN,Time and Material,...,-29.43,-29.43,2.44,2.58,2.44,383.96,151.95,I,1306653,TOTALRETIA Pester Burn Pond
80956,2016-03-01,30018361,ADVCLIENT2462,TOTAL/RETIA - Pester Burn Pond,NaN,NaN,Open/Active,10239122.0,NaN,Time and Material,...,-31.89,-31.89,2.44,2.48,2.44,1273.57,151.95,I,1306653,TOTALRETIA Pester Burn Pond
112204,2016-04-01,30018361,ADVCLIENT2462,TOTAL/RETIA - Pester Burn Pond,NaN,NaN,Open/Active,10239122.0,NaN,Time and Material,...,-33.81,-33.81,2.44,2.63,2.44,2328.18,151.95,I,1306653,TOTALRETIA Pester Burn Pond
144729,2016-05-01,30018361,ADVCLIENT2462,TOTAL/RETIA - Pester Burn Pond,NaN,NaN,Open/Active,10239122.0,NaN,Time and Material,...,-14.97,-14.97,2.44,2.59,2.44,1334.96,151.95,I,1306653,TOTALRETIA Pester Burn Pond
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656582,2020-11-01,30018361,3.00001e+14,OK001557.2018-TOTAL/RETIA - Pester Burn Pond,Plan Required,NaN,Closed,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,OKTOTALRETIA Pester Burn Pond
1656583,2020-12-01,30018361,3.00001e+14,OK001557.2018-TOTAL/RETIA - Pester Burn Pond,Plan Required,NaN,Closed,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,OKTOTALRETIA Pester Burn Pond
1656584,2021-01-01,30018361,3.00001e+14,OK001557.2018-TOTAL/RETIA - Pester Burn Pond,Plan Required,NaN,Closed,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,OKTOTALRETIA Pester Burn Pond
1656585,2021-02-01,30018361,3.00001e+14,OK001557.2018-TOTAL/RETIA - Pester Burn Pond,Plan Required,NaN,Closed,NaN,NaN,Time and Material with cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,OKTOTALRETIA Pester Burn Pond


In [325]:
df_tomerge =result[result['Fiscal Period Code'] < '2019-07-01']
df_tomerge2 =result[result['Fiscal Period Code'] > '2019-06-01']

In [326]:
df_tomerge['Project_Number'] = df_tomerge['Project_Number'].map(legacydict).fillna(df_tomerge['Project_Number'])

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [294]:
#df12_uniquelegacy['legacycleaned'].nunique() freq=df.groupby('Project Number')


14496

In [327]:
df_tomerge['Project_Number'] = df_tomerge['Project_Number'].astype('str') 

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [328]:
df_tomerge['Project_Number'].nunique()

40608

In [307]:
#result.to_csv('results.csv')

In [310]:
result.columns

Index(['Fiscal Period Code', 'Project_Number', 'Project Id', 'Project_Name',
       'Health and Safety Status', 'project_date_create', 'Project Status',
       'SalesCloudClientNo', 'Related Opportunity Project Number',
       'Contract Type Name', 'Ultimate Customer AUS Market Sector',
       'Division Short Name', 'location_2', 'location_4',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Funding Amount - ITC (Snapshot)', 'Backlog NR',
       'Actual Direct Labor Cost - MTD', 'Actual Direct Labor Cost - YTD',
       'Actual Gross Revenue - ITD', 'Actual Gross Revenue - MTD',
       'Actual Gross Revenue - YTD', 'Actual Net Revenue - ITD',
       'Actual Net Revenue - MTD', 'Actual Net Revenue - YTD', 'EBITA - MTD',
       'EBITA - YTD', 'EBITA - ITD', 'Forecast Direct Labor Cost - ITC',
       'Forecast Net Revenue - ITC', 'Order Intake (NR) - MTD',
       'Order Intake (NR) - QTD', 'Order Intake (NR) - YTD', 'WBS1',
       'bookings_jtd', 'interest_curren

In [333]:
df_clean=resulting.groupby('Project_Number').count()
#df_clean.pop('Project Number')


#This should be the names with more than two numbers
df_clean.reset_index(inplace=True)
df_clean.sort_values('count', 
                      inplace=True, 
                      ascending=False)
df_clean

,Project_Number,Fiscal Period Code,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,interest_jtd,interest_ytd,multiplier_budget,multiplier_current,multiplier_target,overhead_current,overhead_rate,revenue_type,count,Cleaned Name
63845,nan,493998,493998,493998,0,0,493998,493704,0,493041,...,493998,493998,493998,493998,493998,493998,493998,493998,493998,493998
46190,DE0007120042,237,231,237,0,0,237,237,0,236,...,231,231,231,231,231,231,231,231,237,237
18294,30017452,235,235,235,0,0,235,235,0,235,...,235,235,235,235,235,235,235,235,235,235
14708,30013830,217,211,217,21,0,217,202,0,217,...,196,196,196,196,196,196,196,196,217,217
60705,TNTVASQNGW18,160,160,160,0,0,160,160,0,160,...,160,160,160,160,160,160,160,160,160,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39982,30083800,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,1
39981,30083799,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,1
39980,30083758,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,1
36708,30065606,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,1


In [335]:
len(df_clean[df_clean['count'] > 64])

149

In [331]:
df_tomerge2 =result[result['Fiscal Period Code'] > '2019-06-01']

In [332]:
frames = [df_tomerge, df_tomerge2]
resulting = pd.concat(frames)

In [336]:
len(resulting)

1481968

In [337]:
resulting.to_csv('resulting.csv')